In [1]:
import pandas as pd
import numpy as np
import datetime, random

In [2]:
df = pd.read_csv('inpatient 2014-2018 creatinine.csv') #1441707, 4
df.pat_mrn_id = df.pat_mrn_id.str.strip('MR').astype('int')
df.time = pd.to_datetime(df.time)
print(df.dtypes)
df.columns = ['mrn',
              'enc',
              'time',
              'creat']
df.set_index(['mrn', 'enc'], inplace=True)

pat_mrn_id                 int32
pat_enc_csn_id             int64
time              datetime64[ns]
creatinine               float64
dtype: object


In [3]:
#Quick question to ask: say a patient comes in on a separate encounter within 7 days
#and their creatinine has gone up by 50% since their previous encounter. Does that count as AKI?

#((df.groupby(['pat_mrn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()
#((df.groupby(['pat_mrn_id', 'pat_enc_csn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()

delta_df = df.groupby(['mrn', 'enc']).shift(-1) - df

In [4]:
df['delta_creat'] = delta_df['creat']
df['delta_time'] = delta_df['time']
#df[['delta_time, delta_creat']] = delta_df[['time', 'creatinine']]
df.delta_creat = df.delta_creat.shift(1)
df.delta_time = df.delta_time.shift(1)

In [5]:
np.where(np.logical_and(df.delta_creat > df.creat.shift(1)*0.5, df.delta_time > datetime.timedelta(days=2)))
#676, 7397, ...
#df.iloc[7390:7400]

(array([    676,    7397,   10250, ..., 1440145, 1440691, 1441349],
       dtype=int64),)

In [6]:
condition1 = np.logical_and(df.delta_creat > 0.3, df.delta_time < datetime.timedelta(hours=48))
condition2 = np.logical_and(df.delta_creat > df.creat.shift(1)*0.5, df.delta_time < datetime.timedelta(days=7))
df['AKI'] = np.logical_or(condition1, condition2)

#Question: Once you've got AKI you've got AKI for the rest of the encounter, right?
# df.loc['MR1000136']

#Condition 1 examples:
# MR1000136, MR1000468, MR1000507

#Condition 2 examples:
#MR1001546, MR1001806, 

#df[np.logical_xor(condition1, condition2)] gives unique examples of C1 or C2